In [1]:
include("src/qsrfe.jl")

Main.qsrfe

In [2]:
using DataFrames
using HyperTuning
using Random
using Distributed

In [3]:

Random.seed!(1234)

TaskLocalRNG()

In [4]:
using LinearAlgebra
using MLJ, MLJLinearModels
X, Y = @load_boston
(xtrain, xtest), (ytrain, ytest) = qsrfe.load_dataset(X,Y;normalize=true,partitioning=0.8,rng=1234);
indexmaptrain = ytrain .<= 47
indexmaptest = ytest .<= 47
(xtrain, xtest), (ytrain, ytest) = (xtrain[indexmaptrain,:], xtest[indexmaptest,:]), (ytrain[indexmaptrain], ytest[indexmaptest]);

In [5]:
size(xtrain), size(xtest)

((389, 12), (98, 12))

In [7]:
using Plots

In [7]:
p1 = Plots.histogram(ytrain, label="train", bins=50)
p2 = Plots.histogram(ytest, label="test", bins=50)
plot(p1, p2, layout=(1,2), size= (1280,480),legend=true)

In [6]:
"2^8:", 2^8, "2^9:", 2^9, "2^10:", 2^10, "2^11:", 2^11, "2^12:", 2^12, "2^13:", 2^13, "2^14:", 2^14, "2^15:", 2^15, "2^16:", 2^16, "2^17:", 2^17

("2^8:", 256, "2^9:", 512, "2^10:", 1024, "2^11:", 2048, "2^12:", 4096, "2^13:", 8192, "2^14:", 16384, "2^15:", 32768, "2^16:", 65536, "2^17:", 131072)

10

In [6]:
# Assemble all the models to test
# Hyper parameter testing cell
N = 2^15
SRFE = qsrfe.srfeRegressor(N=N,σ2=11.0,λ=0.00161612, intercept=true)
RFE = qsrfe.rfeRegressor(N=N,σ2=10.0,λ=0.001, intercept=true)
βQ = qsrfe.βQ(K=2,β=1.5,λ=32,condense=true)
model = SRFE
#training and evaluating
    #quantizaiton

# c = qsrfe.fit(model, xtrain, ytrain,βQ, verbose=true) #,max_iter=2000000
# ytrainpred = qsrfe.predict(model,xtrain,βQ)
# ytestpred = qsrfe.predict(model, xtest,βQ)

    #no quantizaiton
c = qsrfe.fit(model, xtrain, ytrain, verbose=true) #,max_iter=2000000
ytrainpred = qsrfe.predict(model,xtrain)
ytestpred = qsrfe.predict(model, xtest)

println("")
println("test rel: ",qsrfe.rel_L2_error(ytest,ytestpred)," train rel: ",qsrfe.rel_L2_error(ytrain,ytrainpred))
println("test abs: ",qsrfe.abs_error(ytest,ytestpred)," train abs: ",qsrfe.abs_error(ytrain,ytrainpred))
println("test mse: ",qsrfe.mse(ytest,ytestpred)," train mse: ",qsrfe.mse(ytrain,ytrainpred))
println("test rmse: ",qsrfe.rmse(ytest,ytestpred)," train rmse: ",qsrfe.rmse(ytrain,ytrainpred))
model = SRFE
p1 = plot(c[1:(end - model.intercept)],label="c parameter")
p2 = plot(ytrain[sortperm(ytrain)], label="true training values")
plot!(ytrainpred[sortperm(ytrain)], label="SRFE training")
p3 = plot(ytest[sortperm(ytest)], label="true values")
plot!(ytestpred[sortperm(ytest)], label="SRFE testing")
p4 = plot(p1,p3,layout=2, size = (1000,400))
plot(p4,p2,layout=(2,1) ,size=(1000,800))

support: 6148/32769
test rel: 0.12068745220340912 train rel: 0.05727852744767725
test abs: 2.0661405273938453 train abs: 0.9437898383823745
test mse: 8.579310751731064 train mse: 1.643249102189199
test rmse: 2.92904604807283 train rmse: 1.2818927810816312


UndefVarError: UndefVarError: `plot` not defined

In [8]:
# normal linear regression for comparison
lambda = 0.0000002
ridge = RidgeRegression(lambda; fit_intercept=true)

#A = hcat(xtrain)
A = hcat(xtrain,xtrain.^2,xtrain.^3)
# normal linear regression
# c = A \ ytrain ;
# # normal ridge regression
# lambda = 0.00001
# cRidge = (A' * A + lambda * I) \ (A'*ytrain)
c = MLJLinearModels.fit(ridge,A,ytrain)

#ytrainpred = hcat(xtrain,ones(size(xtrain,1))) * c
ytrainpred = hcat(xtrain,xtrain.^2,xtrain.^3,ones(size(xtrain,1))) * c

ytestpred = hcat(xtest,xtest.^2,xtest.^3,ones(size(xtest,1))) * c
println("")
println("test rel: ",qsrfe.rel_L2_error(ytest,ytestpred)," train rel: ",qsrfe.rel_L2_error(ytrain,ytrainpred))
println("test abs: ",qsrfe.abs_error(ytest,ytestpred)," train abs: ",qsrfe.abs_error(ytrain,ytrainpred))
println("test mse: ",qsrfe.mse(ytest,ytestpred)," train mse: ",qsrfe.mse(ytrain,ytrainpred))
# model = RFE
# p1 = plot(c[1:(end - model.intercept)],label="c parameter")
# p2 = plot(ytrain[sortperm(ytrain)], label="true training values")
# plot!(ytrainpred[sortperm(ytrain)], label="SRFE training")
# p3 = plot(ytest[sortperm(ytest)], label="true values")
# plot!(ytestpred[sortperm(ytest)], label="SRFE testing")
# p4 = plot(p1,p3,layout=2, size = (1000,400))
# plot(p4,p2,layout=(2,1) ,size=(1000,800))


test rel: 0.15796017282576505 train rel: 0.14240007206879143
test abs: 2.917169991953711 train abs: 2.383361431117994
test mse: 14.696819518048793 train mse: 10.156407867085619


In [ ]:
quant = βQ(K=256,β=1.5,λ=32,condense=true)
model = srfeRegressor(N=N,λ=λ, σ2=11.0)
trials = 5
N = [2^8, 2^9, 2^10, 2^11, 2^12, 2^13, 2^14, 2^15]
λ = 0.00012
# level, 
df = qsrfe.create_df()
for n in N
    print("current iter: ",n,": ")
    model = srfeRegressor(N=n,λ=λ, σ2=10.0)
    reltesterror, reltrainerror, abstesterror, abstrainerror, msetesterror, msetrainerror, relmsetesterror, relmsetrainerror = trainandevaluate(model,quant,(xtrain, xtest), (ytrain, ytest);trials=trials)
    println("avg. test error: ",reltesterror, " avg. train error: ", reltrainerror)
    println("abs. test error:", abstesterror, " abs. train error: ", abstrainerror)
    println("mse. test error:", msetesterror, " mse train error: ", msetrainerror)
    println("rel mse test error:", relmsetesterror, " rel mse train error: ", relmsetrainerror)
end

In [57]:

function objective(trial)
    @unpack λ = trial
    model = qsrfe.srfeRegressor(N=2^15,λ=λ, σ2=11.0,intercept=true)
    c = qsrfe.fit(model,xtrain,ytrain;max_iter=2000000,verbose=false)
    ytestpred = qsrfe.predict(model,xtest)
    return qsrfe.mse(ytest,ytestpred)
end

scenario = Scenario(λ = (0.0001..0.01),
                        max_trials = 10);

HyperTuning.optimize(objective, scenario)

In [55]:
println("2^8:", 2^8, " 2^9:", 2^9, " 2^10:", 2^10, " 2^11:", 2^11, " 2^12:", 2^12, " 2^13:", 2^13, " 2^14:", 2^14, " 2^15:", 2^15, " 2^16:", 2^16, " 2^17:", 2^17)
levels = [2^10, 2^11, 2^12, 2^13, 2^14, 2^15]
hyperopttrials = 50
trials = 10

2^8:256 2^9:512 2^10:1024 2^11:2048 2^12:4096 2^13:8192 2^14:16384 2^15:32768 2^16:65536 2^17:131072


10

In [26]:
# setup dataframe for plotting later
# level, rel L2, abs, mse
df_SRFE = qsrfe.create_df([0.0, 0.0, 0.0, 0.0])
df = df_SRFE
#first hyper opt finding
for level in levels
    print("current iter: ",level,": ")
    function objective(trial)
        @unpack λ = trial
        model = qsrfe.srfeRegressor(N=level,λ=λ, σ2=11.0,intercept=true)
        c = qsrfe.fit(model,xtrain,ytrain;max_iter=2000000,verbose=false)
        ytestpred = qsrfe.predict(model,xtest)
        return qsrfe.mse(ytest,ytestpred)
    end

    scenario = Scenario(λ = (0.0001..0.01),
                            max_trials = hyperopttrials);
    HyperTuning.optimize(objective, scenario)
    @unpack λ = scenario
    λ
    meanL2 = 0.0
    meanabs = 0.0
    meanmse = 0.0
    for i in 1:trials
        model = qsrfe.srfeRegressor(N=level,λ=λ, σ2=11.0,intercept=true)
        c = qsrfe.fit(model,xtrain,ytrain;max_iter=2000000,verbose=false)
        ytestpred = qsrfe.predict(model,xtest)
        meanL2 = meanL2 + qsrfe.mse(ytest,ytestpred)/trials
        meanabs = meanabs + qsrfe.abs_error(ytest,ytestpred)/trials
        meanmse = meanmse + qsrfe.mse(ytest,ytestpred)/trials

    end
    push!(df[level, meanL2, meanabs, meanmse])
    println("avg. test error: ",meanmse)
end

0.0051621412746435955